In [17]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
import chromadb
import numpy as np
from typing import List, Dict, Any, Tuple, Optional
from groq import Groq
from google.colab import userdata
from datetime import datetime
import re
import json

In [ ]:
class RAGChatbot:
    def __init__(self,
                 chroma_persist_dir: str = "/content/chroma_db",
                 chroma_collection_name: str = "facom_regulamento",
                 bi_encoder_name: str = "paraphrase-multilingual-MiniLM-L12-v2",
                 cross_encoder_name: str = "cross-encoder/ms-marco-MiniLM-L-6-v2",
                 groq_model: str = "llama-3.3-70b-versatile",
                 device: str = "cpu",
                 enable_fact_score: bool = False):

        self.client = chromadb.PersistentClient(path=chroma_persist_dir)
        self.collection = self.client.get_collection(chroma_collection_name)

        self.bi_encoder = SentenceTransformer(bi_encoder_name, device=device)
        self.cross_encoder = CrossEncoder(cross_encoder_name, device=device)

        self.groq_client = Groq(api_key="")
        self.groq_model = groq_model

        self.conversation_history = []
        self.total_tokens = 0
        self.enable_fact_score = enable_fact_score
        self.fact_scores_history = []

        print(f"🤖 Chatbot inicializado!")
        print(f"📚 Base: {chroma_collection_name} ({self.collection.count()} documentos)")
        print(f"🧠 Modelo: {groq_model}")
        print(f"✓ Fact Score: {'Ativado' if enable_fact_score else 'Desativado'}\n")

    def retrieve_candidates(self, query: str, top_k: int = 20) -> Dict[str, Any]:
        q_vec = self.bi_encoder.encode([query], convert_to_numpy=True)[0].tolist()
        res = self.collection.query(
            query_embeddings=[q_vec],
            n_results=top_k,
            include=["documents", "distances", "metadatas"]
        )

        return {
            "ids": res["ids"][0],
            "docs": res["documents"][0],
            "distances": res["distances"][0],
            "metadatas": res.get("metadatas", [None])[0]
        }

    def rerank(self, query: str, docs: List[str]) -> List[Tuple[float, int]]:
        if not docs:
            return []
        pairs = [[query, d] for d in docs]
        scores = self.cross_encoder.predict(pairs)
        scored = [(float(s), int(i)) for i, s in enumerate(scores)]
        return sorted(scored, key=lambda x: x[0], reverse=True)

    def retrieve_and_rerank(self, query: str, retrieve_k: int = 20, final_k: int = 3) -> List[Dict[str, Any]]:
        candidates = self.retrieve_candidates(query, top_k=retrieve_k)

        if not candidates["docs"]:
            return []

        reranked = self.rerank(query, candidates["docs"])

        results = []
        for score, idx in reranked[:final_k]:
            results.append({
                "text": candidates["docs"][idx],
                "score": float(score),
                "metadata": candidates["metadatas"][idx]
            })

        return results

    def decompose_into_facts(self, answer: str) -> List[str]:
        prompt = f"""Decomponha a resposta abaixo em afirmações atômicas independentes.
          Cada afirmação deve ser uma unidade de informação que pode ser verificada individualmente.

          REGRAS:
          - Uma afirmação por linha
          - Cada afirmação deve ser auto-contida
          - Não adicione informações que não estão na resposta
          - Numere cada afirmação (1., 2., 3., etc.)

          RESPOSTA A DECOMPOR:
          {answer}

          AFIRMAÇÕES ATÔMICAS:"""

        response = self.groq_client.chat.completions.create(
            model=self.groq_model,
            messages=[
                {"role": "system", "content": "Você é um especialista em decomposição de texto em afirmações atômicas."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=1024
        )

        decomposition = response.choices[0].message.content

        facts = []
        for line in decomposition.split('\n'):
            line = line.strip()
            match = re.match(r'^\d+\.\s*(.+)', line)
            if match:
                facts.append(match.group(1).strip())

        return facts

    def verify_fact(self, fact: str, context_docs: List[str]) -> Dict[str, Any]:
        context = "\n\n".join([f"[Documento {i+1}]\n{doc}" for i, doc in enumerate(context_docs)])

        prompt = f"""Você deve verificar se a AFIRMAÇÃO abaixo é suportada pelos DOCUMENTOS fornecidos.

          DOCUMENTOS:
          {context}

          AFIRMAÇÃO:
          {fact}

          Responda APENAS com um JSON no seguinte formato (sem markdown, sem explicações extras):
          {{
              "supported": true ou false,
              "confidence": "high", "medium" ou "low",
              "explanation": "breve explicação de 1-2 frases"
          }}

          IMPORTANTE:
          - "supported": true apenas se a afirmação está claramente nos documentos
          - "supported": false se contradiz ou não está nos documentos
          - "confidence": seu nível de certeza na verificação"""

        response = self.groq_client.chat.completions.create(
            model=self.groq_model,
            messages=[
                {"role": "system", "content": "Você é um verificador de fatos preciso. Responda APENAS com JSON válido."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=256
        )

        result_text = response.choices[0].message.content.strip()
        result_text = re.sub(r'```json\s*|\s*```', '', result_text).strip()

        try:
            result = json.loads(result_text)
            return result
        except json.JSONDecodeError:
            return {
                "supported": False,
                "confidence": "low",
                "explanation": "Erro ao processar verificação"
            }

    def calculate_fact_score(self, answer: str, context_docs: List[str]) -> Dict[str, Any]:
        facts = self.decompose_into_facts(answer)

        if not facts:
            return {
                "fact_score": 1.0,
                "total_facts": 0,
                "supported_facts": 0,
                "details": []
            }

        verifications = []
        supported_count = 0

        for fact in facts:
            verification = self.verify_fact(fact, context_docs)
            verification['fact'] = fact
            verifications.append(verification)

            if verification['supported']:
                supported_count += 1

        fact_score = supported_count / len(facts) if facts else 0.0

        return {
            "fact_score": round(fact_score, 3),
            "total_facts": len(facts),
            "supported_facts": supported_count,
            "unsupported_facts": len(facts) - supported_count,
            "details": verifications
        }

    def chat(self,
             user_message: str,
             use_rag: bool = True,
             retrieve_k: int = 20,
             final_k: int = 3,
             temperature: float = 0.5,
             max_tokens: int = 1024,
             calculate_fact_score: bool = None) -> Dict[str, Any]:

        if calculate_fact_score is None:
            calculate_fact_score = self.enable_fact_score

        self.conversation_history.append({
            "role": "user",
            "content": user_message,
            "timestamp": datetime.now().strftime("%H:%M:%S")
        })

        messages = []
        context_docs_list = []

        system_prompt = """Você é um assistente virtual especializado no regulamento acadêmico da FACOM/UFMS (Faculdade de Computação da Universidade Federal de Mato Grosso do Sul).

          Características:
          - Amigável e prestativo
          - Responde em português brasileiro
          - Usa linguagem clara e acessível
          - Quando usa RAG, baseia-se apenas nas informações fornecidas
          - Quando não usa RAG, pode conversar naturalmente sobre tópicos gerais
          - Admite quando não sabe algo
          - Mantém contexto da conversa anterior"""

        messages.append({"role": "system", "content": system_prompt})

        if use_rag:
            context_docs = self.retrieve_and_rerank(user_message, retrieve_k, final_k)

            if context_docs:
                context_docs_list = [doc['text'] for doc in context_docs]
                context = "\n\n".join([
                    f"[Documento {i+1}]\n{doc['text']}"
                    for i, doc in enumerate(context_docs)
                ])

                rag_instruction = f"""CONTEXTO DO REGULAMENTO:
                  {context}

                  Use as informações acima para responder a pergunta do usuário. Se a informação não estiver no contexto, diga que não encontrou."""

                messages.append({"role": "system", "content": rag_instruction})

        recent_history = self.conversation_history[-7:-1]
        for msg in recent_history:
            messages.append({
                "role": msg["role"],
                "content": msg["content"]
            })

        messages.append({"role": "user", "content": user_message})

        response = self.groq_client.chat.completions.create(
            model=self.groq_model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )

        assistant_message = response.choices[0].message.content
        tokens_used = response.usage.total_tokens
        self.total_tokens += tokens_used

        fact_score_result = None
        if calculate_fact_score and use_rag and context_docs_list:
            fact_score_result = self.calculate_fact_score(assistant_message, context_docs_list)
            self.fact_scores_history.append({
                "question": user_message,
                "answer": assistant_message,
                "fact_score": fact_score_result['fact_score'],
                "timestamp": datetime.now().strftime("%H:%M:%S")
            })

        self.conversation_history.append({
            "role": "assistant",
            "content": assistant_message,
            "timestamp": datetime.now().strftime("%H:%M:%S"),
            "tokens": tokens_used,
            "fact_score": fact_score_result['fact_score'] if fact_score_result else None
        })

        return {
            "answer": assistant_message,
            "tokens": tokens_used,
            "fact_score": fact_score_result
        }

    def clear_history(self):
        self.conversation_history = []
        self.total_tokens = 0
        self.fact_scores_history = []
        print("🗑️  Histórico limpo!")

    def show_history(self):
        if not self.conversation_history:
            print("Nenhuma mensagem no histórico.")
            return

        print("\n" + "="*80)
        print("HISTÓRICO DE CONVERSAÇÃO")
        print("="*80 + "\n")

        for msg in self.conversation_history:
            role = "👤 Você" if msg["role"] == "user" else "🤖 Assistente"
            time = msg.get("timestamp", "")
            tokens = f" ({msg['tokens']} tokens)" if "tokens" in msg else ""
            fact_score = f" [Fact Score: {msg.get('fact_score', 'N/A')}]" if msg.get('fact_score') is not None else ""

            print(f"{role} [{time}]{tokens}{fact_score}")
            print(msg["content"])
            print("-"*80 + "\n")

        print(f"Total de tokens usados: {self.total_tokens}")

    def show_fact_scores(self):
        if not self.fact_scores_history:
            print("Nenhum Fact Score calculado ainda.")
            return

        print("\n" + "="*80)
        print("HISTÓRICO DE FACT SCORES")
        print("="*80 + "\n")

        for i, entry in enumerate(self.fact_scores_history, 1):
            print(f"[{i}] {entry['timestamp']} - Score: {entry['fact_score']:.3f}")
            print(f"Q: {entry['question'][:70]}...")
            print(f"A: {entry['answer'][:70]}...")
            print("-"*80 + "\n")

        avg_score = sum(e['fact_score'] for e in self.fact_scores_history) / len(self.fact_scores_history)
        print(f"📊 Fact Score médio: {avg_score:.3f}")

    def get_stats(self):
        stats = {
            "total_messages": len(self.conversation_history),
            "user_messages": len([m for m in self.conversation_history if m["role"] == "user"]),
            "assistant_messages": len([m for m in self.conversation_history if m["role"] == "assistant"]),
            "total_tokens": self.total_tokens
        }

        if self.fact_scores_history:
            avg_fact_score = sum(e['fact_score'] for e in self.fact_scores_history) / len(self.fact_scores_history)
            stats["avg_fact_score"] = round(avg_fact_score, 3)
            stats["total_fact_scores"] = len(self.fact_scores_history)

        return stats

In [21]:
def run_chatbot(chroma_persist_dir: str = "/content/chroma_db",
                chroma_collection_name: str = "facom_regulamento",
                enable_fact_score: bool = False):

    chatbot = RAGChatbot(
        chroma_persist_dir=chroma_persist_dir,
        chroma_collection_name=chroma_collection_name,
        device="cpu",
        enable_fact_score=enable_fact_score
    )

    print("="*80)
    print("🎓 CHATBOT - REGULAMENTO FACOM/UFMS")
    print("="*80)
    print("\nComandos especiais:")
    print("  /sair          - Encerrar o chat")
    print("  /limpar        - Limpar histórico")
    print("  /historico     - Ver histórico completo")
    print("  /stats         - Ver estatísticas")
    print("  /factscores    - Ver histórico de Fact Scores")
    print("  /norag         - Próxima pergunta SEM buscar no regulamento")
    print("  /factscore     - Calcular Fact Score na próxima resposta")
    print("\n" + "="*80 + "\n")

    use_rag_next = True
    calc_fact_score_next = None

    while True:
        try:
            user_input = input("👤 Você: ").strip()

            if not user_input:
                continue

            if user_input.lower() == "/sair":
                print("\n👋 Até logo! Espero ter ajudado.")
                stats = chatbot.get_stats()
                print(f"📊 Estatísticas: {stats['user_messages']} perguntas, {stats['total_tokens']} tokens usados")
                if 'avg_fact_score' in stats:
                    print(f"   Fact Score médio: {stats['avg_fact_score']}")
                break

            elif user_input.lower() == "/limpar":
                chatbot.clear_history()
                continue

            elif user_input.lower() == "/historico":
                chatbot.show_history()
                continue

            elif user_input.lower() == "/factscores":
                chatbot.show_fact_scores()
                continue

            elif user_input.lower() == "/stats":
                stats = chatbot.get_stats()
                print(f"\n📊 Estatísticas:")
                print(f"   - Total de mensagens: {stats['total_messages']}")
                print(f"   - Suas perguntas: {stats['user_messages']}")
                print(f"   - Respostas do bot: {stats['assistant_messages']}")
                print(f"   - Tokens usados: {stats['total_tokens']}")
                if 'avg_fact_score' in stats:
                    print(f"   - Fact Score médio: {stats['avg_fact_score']}")
                    print(f"   - Total de avaliações: {stats['total_fact_scores']}")
                print()
                continue

            elif user_input.lower() == "/norag":
                use_rag_next = False
                print("✓ Próxima resposta será SEM busca no regulamento\n")
                continue

            elif user_input.lower() == "/factscore":
                calc_fact_score_next = True
                print("✓ Fact Score será calculado na próxima resposta\n")
                continue

            print("🤖 Assistente: ", end="", flush=True)

            result = chatbot.chat(
                user_input,
                use_rag=use_rag_next,
                temperature=0.5,
                calculate_fact_score=calc_fact_score_next
            )

            print(result['answer'])

            if result['fact_score']:
                fs = result['fact_score']
                print(f"\n📊 Fact Score: {fs['fact_score']:.3f} ({fs['supported_facts']}/{fs['total_facts']} afirmações suportadas)")

            print()

            use_rag_next = True
            calc_fact_score_next = None

        except KeyboardInterrupt:
            print("\n\n👋 Chat interrompido. Até logo!")
            break
        except Exception as e:
            print(f"\n❌ Erro: {e}\n")

In [22]:
run_chatbot()

🤖 Chatbot inicializado!
📚 Base: facom_regulamento (62 documentos)
🧠 Modelo: llama-3.3-70b-versatile
✓ Fact Score: Desativado

🎓 CHATBOT - REGULAMENTO FACOM/UFMS

Comandos especiais:
  /sair          - Encerrar o chat
  /limpar        - Limpar histórico
  /historico     - Ver histórico completo
  /stats         - Ver estatísticas
  /factscores    - Ver histórico de Fact Scores
  /norag         - Próxima pergunta SEM buscar no regulamento
  /factscore     - Calcular Fact Score na próxima resposta


👤 Você: Quando o regulamento entra em vigor?
🤖 Assistente: De acordo com o Art. 3º do regulamento, ele entra em vigor em 1º de janeiro de 2026. Essa informação está presente em ambos os Documentos 1 e 2.

👤 Você: /factscore
✓ Fact Score será calculado na próxima resposta

👤 Você: Quando o regulamento entra em vigor?
🤖 Assistente: O regulamento entra em vigor em 1º de janeiro de 2026, conforme estabelecido no Art. 3º.

📊 Fact Score: 1.000 (2/2 afirmações suportadas)

👤 Você: O que o Doutorado P